In [6]:
import pickle
import os
import torch

file_path = "all_h_all.pkl"

# Check if the file exists before loading
if os.path.exists(file_path):
    with open(file_path, "rb") as file:
        data = pickle.load(file)  # Load the pickle file
        print("File loaded successfully!")
else:
    print(f"Error: File '{file_path}' not found!")


File loaded successfully!


In [13]:
len(data[0][0][0])

3584

In [ ]:
num_query = len(data) #0D
num_reasoning = len(data[0]) #1D
num_layer = len(data[0][0]) #2D
num_hidden = len(data[0][0][0]) #3D

###
print("num_query")
num_query = len(data)
print(num_query)
###
print("num_reasoning")
for i in range(num_query):
    num_reasoning = len(data[i])
    print(num_reasoning)

###
print("num_layer")
for i in range(num_reasoning):
    num_layer = len(data[i])
    print(num_reasoning)

4
3
4
5
4
4
2
17
7
4


In [64]:
num_reasoning

4

In [ ]:
torch.stack()

In [46]:
num_hidden

3584

In [ ]:
query0_data = data[0]

for i in range(num_reasoning):
    len(query0_data[i])

(tensor([ 0.0015, -0.0046, -0.0084,  ..., -0.0020, -0.0009, -0.0069]),
 tensor([ 0.0098, -0.1484,  0.0293,  ..., -0.0256,  0.0244, -0.0515]),
 tensor([ 0.0449, -0.1133, -0.0317,  ..., -0.0505, -0.0247, -0.1045]),
 tensor([-0.0693, -0.1260,  0.0520,  ..., -0.0496,  0.0337, -0.0090]),
 tensor([-0.3691, -0.2324,  0.2061,  ..., -0.0933,  0.1138,  0.1001]),
 tensor([-0.4961, -0.3262,  0.1504,  ..., -0.2324,  0.3359,  0.0081]),
 tensor([-0.1748, -0.1777, -0.0098,  ..., -0.2988,  0.3848,  0.0444]),
 tensor([-0.2070, -0.1572, -0.0122,  ..., -0.4727,  0.4434,  0.3418]),
 tensor([-0.1309,  0.0742, -0.1084,  ..., -0.9219,  0.3730,  0.0598]),
 tensor([-0.7109, -0.1016, -0.2188,  ..., -1.0781,  0.0449,  0.0098]),
 tensor([ 0.1797,  0.1396, -0.1797,  ..., -0.6250, -0.1553, -0.1060]),
 tensor([-0.0020,  0.0596,  0.1123,  ..., -0.5742,  0.0586,  0.3105]),
 tensor([ 0.2598,  0.1465,  0.1045,  ..., -0.3145,  0.2441,  0.1230]),
 tensor([ 0.2520, -0.3340,  0.3516,  ..., -0.6797, -0.3750,  0.2598]),
 tenso